In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from alpha_vantage.timeseries import TimeSeries
import datetime as dt
import copy
import matplotlib.pyplot as plt
import time

In [2]:

def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2['ATR']

def CAGR(DF):
    "function to calculate the Cumulative Annual Growth Rate of a trading strategy"
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    n = len(df)/(252*345)
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    vol = df["ret"].std() * np.sqrt(252*345)
    return vol

def sharpe(DF,rf):
    "function to calculate sharpe ratio ; rf is the risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr
    

def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

In [3]:
tickers = ["MSFT","AAPL"]
         

ts = TimeSeries(key="J04L0E3DP2SR0N3P", output_format='pandas')

ohlc_intraday = {} # directory with ohlc value for each stock   
api_call_count = 1
ts = TimeSeries(key="J04L0E3DP2SR0N3P", output_format='pandas')
start_time = time.time()
try:
    
    for ticker in tickers:
        print("getting data for: ",ticker)
        data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='full')[0]
        api_call_count+=1
        data.columns = ["Open","High","Low","Close","Volume"]
        data = data.iloc[::-1]
        data = data.between_time('09:30', '15:15') #remove data outside regular trading hours
        ohlc_intraday[ticker] = data
        if api_call_count==2:
            api_call_count = 1
            time.sleep(60 - ((time.time() - start_time) % 60.0))
except:
    print("moving to next iteration.......error")        

tickers = ohlc_intraday.keys()


getting data for:  MSFT
getting data for:  AAPL
moving to next iteration.......error


In [19]:
ohlc_intraday["MSFT"]

,Open,High,Low,Close,Volume
date,,,,,
2021-05-25 09:30:00,251.7000,251.890,251.60,251.6500,4800.0
2021-05-25 09:31:00,251.7690,251.769,251.34,251.6100,481649.0
2021-05-25 09:32:00,251.6200,252.250,251.62,252.1500,112669.0
2021-05-25 09:33:00,252.1700,252.750,252.13,252.4700,127702.0
2021-05-25 09:34:00,252.4800,252.630,252.30,252.3200,96815.0
...,...,...,...,...,...
2021-06-07 15:11:00,252.6000,252.630,252.55,252.5900,19663.0
2021-06-07 15:12:00,252.5977,252.600,252.54,252.5624,22603.0
2021-06-07 15:13:00,252.5686,252.615,252.56,252.6100,21110.0


In [20]:
ohlc_dict=copy.deepcopy(ohlc_intraday)

In [26]:
for ticker in tickers:
    ohlc_dict[ticker]["o-c"]=abs(ohlc_dict[ticker]["Open"]-ohlc_dict[ticker]["Close"])
    ohlc_dict[ticker]["o-c avg"]=ohlc_dict[ticker]["o-c"].rolling(20).mean()
    ohlc_dict[ticker]["avg volume"]=ohlc_dict[ticker]["Volume"].rolling(20).mean()
    ohlc_dict[ticker].dropna(inplace=True)
    ticker_ret[ticker] = []


In [27]:
ohlc_dict["MSFT"]

,Open,High,Low,Close,Volume,o-c,o-c avg,avg volume
date,,,,,,,,
2021-05-25 10:27:00,251.1850,251.195,250.950,250.9600,32746.0,0.2250,0.097725,32192.15
2021-05-25 10:28:00,250.9500,251.060,250.950,251.0600,26808.0,0.1100,0.102225,31622.00
2021-05-25 10:29:00,251.0500,251.080,250.990,251.0300,38188.0,0.0200,0.094225,31633.50
2021-05-25 10:30:00,251.0200,251.180,251.000,251.1800,29991.0,0.1600,0.092725,30990.00
2021-05-25 10:31:00,251.1800,251.250,251.123,251.1700,26617.0,0.0100,0.088975,30350.95
...,...,...,...,...,...,...,...,...
2021-06-07 15:11:00,252.6000,252.630,252.550,252.5900,19663.0,0.0100,0.048325,25667.60
2021-06-07 15:12:00,252.5977,252.600,252.540,252.5624,22603.0,0.0353,0.049340,25959.50
2021-06-07 15:13:00,252.5686,252.615,252.560,252.6100,21110.0,0.0414,0.048505,26182.65


In [28]:
tickers_signal = {}
ticker_ret = {}


In [32]:
ticker_ret["MSFT"]=[]
signal=""
for ticker in tickers:
    for i in range(len(ohlc_dict[ticker])):
        if signal=="":
            if ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["avg volume"][i-1] and ohlc_dict[ticker]["Open"][i]<ohlc_dict[ticker]["Close"][i] and ohlc_dict[ticker]["o-c"][i]>ohlc_dict[ticker]["o-c avg"][i-1]:
                signal="buy"
                price=ohlc_dict[ticker]["Close"][i]


            elif ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["avg volume"][i-1] and ohlc_dict[ticker]["Open"][i]>ohlc_dict[ticker]["Close"][i] and ohlc_dict[ticker]["o-c"][i]>ohlc_dict[ticker]["o-c avg"][i-1]:
                signal="sell"
            ticker_ret[ticker].append(0)

        if signal=="buy":
            if ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["avg volume"][i-1] and ohlc_dict[ticker]["Open"][i]>ohlc_dict[ticker]["Close"][i] and ohlc_dict[ticker]["o-c"][i]>ohlc_dict[ticker]["o-c avg"][i-1]:

                signal="sell"
                ticker_ret[ticker].append((ohlc_dict[ticker]["Close"][i]/ohlc_dict[ticker]["Close"][i-1])-1)

            elif ohlc_dict[ticker]["Close"][i]>=(price+.02*price) or ohlc_dict[ticker]["Close"][i]<=(price-.02*price):
                ticker_ret[ticker].append((ohlc_dict[ticker]["Close"][i]/ohlc_dict[ticker]["Close"][i-1])-1)
                signal=""
            
            else:
                ticker_ret[ticker].append((ohlc_dict[ticker]["Close"][i]/ohlc_dict[ticker]["Close"][i-1])-1)
           
        
        
        if signal=="sell":
            if ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["avg volume"][i-1] and ohlc_dict[ticker]["Open"][i]<ohlc_dict[ticker]["Close"][i] and ohlc_dict[ticker]["o-c"][i]>ohlc_dict[ticker]["o-c avg"][i-1]:

                signal="buy"
                ticker_ret[ticker].append((ohlc_dict[ticker]["Close"][i-1]/ohlc_dict[ticker]["Close"][i])-1)

            elif ohlc_dict[ticker]["Close"][i]>=(price+.02*price) or ohlc_dict[ticker]["Close"][i]<=(price-.02*price):
                ticker_ret[ticker].append((ohlc_dict[ticker]["Close"][i-1]/ohlc_dict[ticker]["Close"][i])-1)
                signal=""
            
            else:
                ticker_ret[ticker].append((ohlc_dict[ticker]["Close"][i-1]/ohlc_dict[ticker]["Close"][i])-1)

    ohlc_dict[ticker]["ret"] = np.array(ticker_ret[ticker])    

ValueError: Length of values (3110) does not match length of index (3057)